# 數位巴菲特分析機 v0.1
這是原型設計，目的是自動分析具備 Liquid Staking 且低價潛力的加密貨幣。

In [ ]:
import requests
import pandas as pd
import random

token_list = [
    "ethereum", "solana", "staked-ether", "lido-staked-sol",
    "matic-network", "cosmos", "avalanche-2", "polkadot", "aptos", "terra-luna"
]

def fetch_price_data(token_id):
    url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart"
    params = {"vs_currency": "usd", "days": "30"}
    res = requests.get(url, params=params)
    if res.status_code == 200:
        data = res.json()
        prices = [p[1] for p in data['prices']]
        avg_price = sum(prices) / len(prices)
        current_price = prices[-1]
        return round(current_price, 4), round(avg_price, 4)
    else:
        return None, None

def check_lsd_support(token_name):
    lsd_tokens = [
        "steth", "stsol", "msol", "reth", "frxeth", "cbeth", "jitosol", "ankreth",
        "stkatom", "stmatic", "savax"
    ]
    return any(lsd in token_name.lower() for lsd in lsd_tokens)

def eth_relative_ratio(price, eth_price, avg_price, avg_eth_price):
    current_ratio = price / eth_price
    avg_ratio = avg_price / avg_eth_price
    return round((current_ratio - avg_ratio) / avg_ratio * 100, 2)

def simulate_tvl_change(token_name):
    return round(random.uniform(-20, 20), 2)

def compute_score(token_id, price, avg_price, is_lsd, rel_ratio, tvl_change):
    score = 0
    if is_lsd: score += 1
    if price < avg_price: score += 1
    if rel_ratio > 0: score += 1
    if tvl_change > 0: score += 1
    if price < avg_price * 0.9: score += 1
    return score

def analyze_tokens(token_list):
    result = []
    eth_price, eth_avg = fetch_price_data("ethereum")
    for token_id in token_list:
        price, avg_price = fetch_price_data(token_id)
        if not price or not avg_price:
            continue
        is_lsd = check_lsd_support(token_id)
        rel_ratio = eth_relative_ratio(price, eth_price, avg_price, eth_avg)
        tvl_change = simulate_tvl_change(token_id)
        score = compute_score(token_id, price, avg_price, is_lsd, rel_ratio, tvl_change)
        signal = "🟢" if score >= 4 else ("🟡" if score == 3 else "🔴")
        result.append({
            "幣種": token_id,
            "現價($)": price,
            "30日均價($)": avg_price,
            "有LSD?": "✅" if is_lsd else "❌",
            "與ETH價比變化(%)": rel_ratio,
            "TVL變動(%)": tvl_change,
            "總評分": score,
            "訊號": signal
        })
    return pd.DataFrame(result)

df = analyze_tokens(token_list)
df.sort_values(by="總評分", ascending=False).reset_index(drop=True)